---
<div align="center">

# Connect-4 [Artificial Intelligence Project]
</div>

---

<div align="center">

### Search Problem 
</div>

A Search Problem is composed by multiple factors:
- States (Any possible configuration of the Board)
- Initial and Goal States
- Action Space (actions and their effects on the environment)
- Action Cost

---

In [25]:
from Constants import  (NROWS, NCOLS, # Board's Size
                        SQSIZE, X_OFFSET, Y_OFFSET, BORDER_THICKNESS, WIDTH, HEIGHT, CIRCLE_OFFSET, CIRCLE_POS, CIRCLE_RADIUS, # Some Parameters for the Graphical User Interface
                        BLACK, WHITE, LIGHT_BLUE, BLUE, DARK_BLUE, RED, DARK_RED, GREEN, DARK_GREEN, PIECES_COLORS) # RGB Colors
import numpy as np
from copy import (deepcopy)
from time import (time)
import random as rd
from IPython.display import (clear_output) # Helps clear the output of cells without having to do it manually
import heapq
import pygame
import sys

In [32]:
# Defined Usefull Wrapper to Measure how long does a function take to execute - Might be used later to compare algorithm's efficiency

def time_it(function):
    def wrapper(*args, **kwargs):
        start = time()
        result = function(*args, **kwargs)
        print(f"{function.__name__} took {time() - start} seconds")
        return result
    return wrapper

In [33]:
class Connect_Four_State:

    def __init__(self):
        # Matrix to Store the Board's Values
        self.board = np.zeros(shape=(NROWS, NCOLS), dtype=np.int8)

        # Array to keep track of the row's index for every column where a piece can be placed
        # Basically it's the Row idx for every column's height
        self.columns_height = np.full(shape=NCOLS, fill_value=(NROWS - 1), dtype=np.int8)

        # Defining the Possible Actions (Initially a piece can be placed in any column)
        self.actions = [col for col in range(NCOLS)]
        
        # Initializing a variable to track the current player
        self.current_player = 1
        
        # Variable to store the Winner (-1 - Game still running || 0 - Tie || 1 - PLayer 1 || 2 - Player 2 / AI)
        self.winner = -1

        # Setting a varible to store the board's move_history
        self.move_history = [self.board.copy()]
    
    def next_player(self):
        # Returns the next turn's player
        return 3 - self.current_player

    def is_over(self):
        # If the Winner corresponds to -1 then the game is not finished otherwise it is
        return self.winner != -1

    def reset(self):
        # Calls back the Constructor
        return self.__init__()
    
    def inside_board(self, x, y):
        # Checks if a position (x,y) exists inside the board's matrix
        return (x >= 0 and x < NROWS) and (y >= 0 and y < NCOLS)
    
    def move(self, ncol):
        # Creating a new state
        new_state = deepcopy(self)

        # Checks of the given column can be played on
        if(ncol in self.actions):
            # Inserting the move into the board
            new_state.board[new_state.columns_height[ncol]][ncol] = new_state.current_player
            
            # Updating the "ncol"'s height
            new_state.columns_height[ncol] -= 1
            
            # Checking if the column is full and therefore uncapable of receiving more pieces -> Changes the action space
            if (new_state.columns_height[ncol] < 0):
                new_state.actions.remove(ncol)

            # Updates the current Winner
            new_state.update_winner()

            # Updating current player for the next state
            new_state.current_player = new_state.next_player()

            # Updating move_history
            new_state.move_history.append(new_state.board.copy())
        
        # Returns the New State (Could be the same as the previous if the action could not be performed)
        return new_state

    def generate_new_states(self):
        # List to contain all the new states
        new_states = []

        # Iterates through all possible actions and creates a new state for each one
        for ncol in self.actions:
            new_states.append(self.move(ncol))

        # Returns all generated states
        return new_states
    
    def check_line(self, n, player, values):
        # -> Checks for a 4-piece line given 4 consecutive values from the matrix
        # Calculates the number of pieces from the value's array that are from the given player
        player_pieces = sum(list(map(lambda piece: piece == player, values)))
        
        # Finds a Full 4-piece Line
        if n == 4:
            return player_pieces == 4
        
        # Finds a complete 3-piece Line
        if n == 3:
            # Checks if it's possible to make it a 4-piece Line by adding another
            empty_places = sum(list(map(lambda piece: piece == 0, values)))
            return player_pieces == 3 and empty_places == 1
    
    def count_lines(self, n, player):
        # -> Searches the Board looking for a 4-piece Combination
        # Initializes the number of lines found
        total_lines = 0

        # Loops through the board
        for row in range(NROWS):
            for col in range(NCOLS):
                # Found a Horizontal Line
                if col < NCOLS - 3 and self.check_line(n, player, [self.board[row][col + i] for i in range(4)]):
                    total_lines += 1
                
                # Found a Vertical Line
                if row < NROWS - 3 and self.check_line(n, player, [self.board[row + i][col] for i in range(4)]):
                    total_lines += 1
                
                # Found a Descending Diagonal Line
                if row < NROWS - 3 and col < NCOLS - 3 and self.check_line(n, player, [self.board[row + i][col + i] for i in range(4)]):
                    total_lines += 1
                
                # Found a Ascending Diagonal Line
                if col < NCOLS - 3 and row > 3 and self.check_line(n, player, [self.board[row - i][col + i] for i in range(4)]):
                    total_lines += 1
        
        return total_lines
    
    def update_winner(self):
        # -> Updates the Current State's Winner
        
        # Checks for a 4-piece combination made by PLayer 1
        if self.count_lines(4, 1) > 0:
            self.winner = 1
        
        # Checks for a 4-piece combination made by PLayer 1
        elif self.count_lines(4, 2) > 0:
            self.winner = 2

        # Checks for Possible moves 
        if (len(self.actions) == 0):
            self.winner = 0

    """ AUXILIAR METHODS """

    def __str__(self):
        # -> Converts the board into the style used in the Assignment 1 Paper
        DECODER = {0:'-', 1:'X', 2:'O'} 
        line = ["-" for i in range(2*NCOLS -1)]
        line.insert(0, '#')
        line.insert(1, ' ')
        line.insert(len(line), ' ')
        line.insert(len(line), '#')
        formated_line = "".join(line)
        new_board = formated_line + '\n'
        for x in range (NROWS):
            for y in range (NCOLS):
                if (y == 0):
                    new_board += "| " + DECODER[self.board[x][y]]
                elif (y == NCOLS -1):
                    new_board += " " + DECODER[self.board[x][y]] + " |"
                else:
                    new_board += " " + DECODER[self.board[x][y]]
            new_board += '\n'
        new_board += formated_line
        return new_board
        
    def __hash__(self):
        return hash(str(self.board))

    def __eq__(self, other:object):
        if (not isinstance(other, Connect_Four_State)):
            raise Exception(f"Sorry, other object is not an instance of {self.__class__.__name__}")
        return hash(self) == hash(other)

In [34]:
game = Connect_Four_State()
print(game, "\n")

game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
game = game.move(1)
print(game)

# ------------- #
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
# ------------- # 

# ------------- #
| - O - - - - - |
| - X - - - - - |
| - O - - - - - |
| - X - - - - - |
| - O - - - - - |
| - X - - - - - |
# ------------- #


---
<div align="center">

## TreeNode Class
</div>

---

In [35]:
class TreeNode:
    def __init__(self, state, parent=None):
        # Stores a State of the Game
        self.state = state

        # Keeps a reference to his Parent Node
        self.parent = parent

        # Stores all the generated nodes
        self.children = []

    def is_leaf(self):
        # Returns the True if the Node does not have ny children and therefore is a leaf
        return len(self.children) == 0

    def generate_new_node(self, ncol):
        # Creates a new state after the move
        new_state = self.state.move(ncol)

        # Wraps it with a TreeNode 
        new_node = TreeNode(state=new_state, parent=self)
        
        # Inserts the New Node into the Current Node's Children
        self.children.append(new_node)
        
        # Returns the generated Node
        return new_node

    def __str__(self):
        return str(self.state)
    
    def __hash__(self):
        return hash(str(self.state) + str(self.parent) + "".join([child for child in self.children]))
    
    def __eq__(self, other:object):
        if (not isinstance(other, TreeNode)):
            raise Exception(f"Sorry, other object is not an instance of {self.__class__.__name__}")
        return self.__hash__() == other.__hash__()

In [36]:
initial_state = Connect_Four_State()
new_node = TreeNode(initial_state)

---
<div align="center">

## Algorithms
</div>

---

### A* Search

In [37]:
def A_Star_Search(initial_node, heuristic):

    # Setting a method in the TreeNode Class - Compares 2 Nodes taking into consideration their state's heuristic as well as the respective path's cost
    setattr(TreeNode, "__lt__", lambda self, other: (heuristic(self.state) + len(self.state.move_history) - 1) < heuristic(other.state) + len(other.state.move_history) - 1)
    
    # Setting the Initial Node
    root = initial_node

    # Initializing a queue to help manage the generated nodes
    queue = [root]

    # Creating a set of visited_states so that we don't waste time generating new_states from an already visited state
    visited_states = set()

    # While we have nodes inside the queue
    while queue:

        # Pop current_node
        current_node = heapq.heappop(queue)

        # Continue if the state was already visited 
        if current_node.state in visited_states:
            continue

        # Updating the visited_states set
        visited_states.add(current_node.state)

        # Checking if we found a Final State [if so return it]
        if current_node.state.is_over():
            return current_node

        # Generating new_states and adding them to the queue (wrapped with a TreeNode) if they were not visited
        for new_state in current_node.state.generate_new_states():
            if (new_state not in visited_states):
                child = TreeNode(state=new_state, parent=current_node)
                heapq.heappush(queue, child)
    
    # If we didn't found a Solution then we return None                
    return None

### MinMax - MAYBE????

In [38]:
def MinMax(node:TreeNode):
    """ MinMax with Alpha-Beta Prunning - EXTRA """
    pass

### Monte Carlo Tree Search

In [39]:
def MCTS(node:TreeNode):
    """ Monte Carlo Tree Search """
    pass

---
<div align="center">

## Heuristics [MAKE MORE HEURISTICS]
</div>

---

In [40]:
""" DEFINED THE SUGGESTED HEURISTIC - NOT 100% SURE [I THINK IT'S CALCULATING SOME PIECES MORE THAN ONCE INSIDE THE SAME LINE TYPE] """

def check_line_configuration(player, values):
    # Given a Array of values if counts the amount of pieces from each type
    player_pieces = sum(list(map(lambda piece: piece == player, values)))
    enemy_player_pieces = sum(list(map(lambda piece: piece == (3 - player), values)))
    empty_spaces = sum(list(map(lambda piece: piece == 0, values)))
    return (player_pieces, enemy_player_pieces, empty_spaces)

def calculate_line_score(player_pieces, enemy_pieces, empty_spaces):
    # -> Calculates the score to return based on the line configuration
    # Defining a Score Decoder for the amount of Empty Spaces
    SCORE_DECODER = {0:512,  # 0 Empty Spaces - There are 4 player's pieces
                     1:50,   # 1 Empty Space  - There are 3 player's pieces
                     2:10,   # 2 Empty Spaces - There are 2 player's pieces
                     3:1,    # 3 Empty Spaces - There is 1 player's pieces
                     4:0}    # 4 Empty Spaces - There are only empty spaces

    if (player_pieces > 0):
        # We have both player's pieces
        if (enemy_pieces > 0):
            return 0
        # There are no enemy pieces
        else:
            return SCORE_DECODER[empty_spaces]
    else:
        # Only have empty pieces
        if (enemy_pieces == 0):
            return 0
        else:
            return - SCORE_DECODER[empty_spaces]

def calculate_score(state):
    # -> Calculates current State Evaluation [Based on the Assignment's Suggestion]
    # Initializes the number of lines found
    total_score = 0

    # Loops through the board
    for row in range(NROWS):
        for col in range(NCOLS):
            # Checks a Horizontal Line
            if col < NCOLS - 3:
                (player_pieces, enemy_player_pieces, empty_spaces) = check_line_configuration(state.current_player, [state.board[row][col + i] for i in range(4)])
                total_score += calculate_line_score(player_pieces, enemy_player_pieces, empty_spaces)
            
            # Checks a Vertical Line
            if row < NROWS - 3:
                (player_pieces, enemy_player_pieces, empty_spaces) = check_line_configuration(state.current_player, [state.board[row + i][col] for i in range(4)])
                total_score += calculate_line_score(player_pieces, enemy_player_pieces, empty_spaces)
            
            # Checks a Descending Diagonal Line
            if row < NROWS - 3 and col < NCOLS - 3:
                (player_pieces, enemy_player_pieces, empty_spaces) = check_line_configuration(state.current_player, [state.board[row + i][col + i] for i in range(4)])
                total_score += calculate_line_score(player_pieces, enemy_player_pieces, empty_spaces)

            # Checks a Ascending Diagonal Line
            if col < NCOLS - 3 and row > 3:
                (player_pieces, enemy_player_pieces, empty_spaces) = check_line_configuration(state.current_player, [state.board[row - i][col + i] for i in range(4)])
                total_score += calculate_line_score(player_pieces, enemy_player_pieces, empty_spaces)
    
    return total_score

---
<div align="center">

## Player's Actions
</div>

---

### Player [Action]

In [41]:
def player_terminal(current_node):
    # Printing current board configuration
    print(f"\n  CURRENT BOARD \n{current_node.state}")
    
    # Requesting a column to play at
    ncol = int(input(f"| Player {current_node.state.current_player} | Choose a Column to Play: "))
    
    # Creating a new Node by making a move into the "ncol" column
    new_node = current_node.generate_new_node(ncol)

    return new_node

def player_GUI(current_node, clicked=False):
    # In case we don't do any move the the node stays the same
    new_node = current_node
    
    # Checking if we pressed the mouse 1 button and therefore changed the self.clicked flag
    if not clicked and pygame.mouse.get_pressed()[0] == 1:
        
        # Getting Mouse Position
        (y, x) = pygame.mouse.get_pos()

        # Modifying the Mouse Coordinates to match with the Screen Stylling
        x = (x - Y_OFFSET) // SQSIZE
        y = (y - X_OFFSET) // SQSIZE

        # Checking if the coordenates exist in the board. If so, add a piece to the column that the mouse was pressed on
        if (current_node.state.inside_board(x, y)):
            new_node = current_node.generate_new_node(y)

        # Updating the "clicked" flag
        clicked = True

    # Checking if we released the mouse 1 button and therefore changed the self.clicked flag
    if clicked and pygame.mouse.get_pressed()[0] == 0:
        
        # Updating the "clicked" flag
        clicked = False

    return (new_node, clicked)

### Random [Action]

In [42]:
def random_terminal(current_node, show=True):
    # Randomizing a col to play at & printing wich one it was
    ncol = rd.randrange(0, len(current_node.state.actions))

    if (show):
        # Printing current board configuration
        print(f"\n  CURRENT BOARD \n{current_node.state}")
        print(f"\n| Random AI | Played in the {ncol}th column ")

    # Creating a new Node by making a move into the "ncol" column
    new_node = current_node.generate_new_node(ncol)

    return new_node

def random_GUI(current_node, clicked=False):
    # Randomizing a col to play at & printing wich one it was
    ncol = rd.randrange(0, len(current_node.state.actions))

    # Creating a new Node by making a move into the "ncol" column
    new_node = current_node.generate_new_node(ncol)

    return (new_node, clicked)

### A* Search [Action]

In [43]:
def A_Star_action(current_node, heuristic):
    # Getting the Final Node after using the A* Search
    final_node = A_Star_Search(current_node, heuristic)

    # Finding next node after the "current_node" inside the "final_node"'s nth parents
    while final_node.parent != current_node:
        final_node = final_node.parent
    
    # return next node
    return final_node

def A_Star_terminal(current_node, heuristic=calculate_score, show=True):
     
    if (show):
        # Printing current board configuration
        print(f"\n  CURRENT BOARD \n{current_node.state}")
        print(f"\n| A* Search AI | Played ")
    
    # Generate the Next_Node
    new_node = A_Star_action(current_node, heuristic)

    # Returns the next node
    return new_node

---
<div align="center">

## Connect_Four [Main Class]
</div>

---

In [44]:
class Connect_Four:
    def __init__(self, player1_tactic, player2_tactic):
        self.current_node = TreeNode(state=Connect_Four_State())

        self.player1 = player1_tactic
        self.player2 = player2_tactic
        self.PLAYER_ACTION = {1:self.player1, 2:self.player2}

        self.clicked = False

    """ TERMINAL METHODS """
    
    def run_terminal(self):
        # Creating the Game Loop inside the Terminal (It Runs while we haven't reached a Final State)
        while not self.current_node.state.is_over():

            # Creating a new Node by using the player's tactic
            new_node = self.PLAYER_ACTION[self.current_node.state.current_player](self.current_node)
            
            # Updating Current Node
            self.current_node = new_node

        # Printing Final Board Configuration
        print(f"\n   FINAL BOARD\n{self.current_node.state}")
        
        # Checking if it was a Tie
        if self.current_node.state.winner == 0:
            print("Tie")
        else: # A Player Won
            print(f"\n-> {self.PLAYER_ACTION[self.current_node.state.winner].__name__} {self.current_node.state.winner} Wins!")

        return self.current_node.state.winner

    """ GUI METHODS """
    
    def write(self, font, text, size, color, bg_color, bold, pos, screen):
        # Writes Text into the Screen
        letra = pygame.font.SysFont(font, size, bold)
        frase = letra.render(text, 1, color, bg_color)
        screen.blit(frase, pos)
    
    def write_winner(self, screen):
        winner_name = " " + self.PLAYER_ACTION[self.current_node.state.winner].__name__ + " " + str(self.current_node.state.winner) + " Wins!"
        font_size = 50
        winner_text = len(winner_name) * font_size
        (x, y) = ((WIDTH - winner_text) // 2, (Y_OFFSET - font_size) // 2)
        self.write(font='Arial', text=winner_name, size=font_size, color=LIGHT_BLUE, bg_color=WHITE, bold=True, pos=(3*x, y), screen=screen)
    
    def draw_board(self, screen):
        # Draws Board's Shadow
        board_rect_shadow = pygame.Rect((X_OFFSET - BORDER_THICKNESS, Y_OFFSET - BORDER_THICKNESS),
                                        (SQSIZE*NCOLS + 2*BORDER_THICKNESS, SQSIZE*NROWS + 2*BORDER_THICKNESS))
        pygame.draw.rect(screen, DARK_BLUE, board_rect_shadow)

        # Draws Main Board
        board_rect = pygame.Rect((X_OFFSET, Y_OFFSET), (SQSIZE*NCOLS, SQSIZE*NROWS))
        pygame.draw.rect(screen, BLUE, board_rect)

        # Drawing Circles in the Board
        for row in range(NROWS):
            for col in range(NCOLS):
                # Getting the Colors from the Auxiliar List
                (Border_Color, Circle_Color) = PIECES_COLORS[self.current_node.state.board[row][col]]

                # Drawing the Board's border around the pieces
                pygame.draw.circle(screen, DARK_BLUE, (X_OFFSET + SQSIZE//2 + (col*SQSIZE),
                                                       Y_OFFSET + SQSIZE//2 + (row*SQSIZE)), int(1.15*CIRCLE_RADIUS))
                
                # Drawing the Circle's Border
                pygame.draw.circle(screen, Border_Color, (X_OFFSET + SQSIZE//2 + (col*SQSIZE),
                                                          Y_OFFSET + SQSIZE//2 + (row*SQSIZE)), CIRCLE_RADIUS)

                # Drawing the Main Circle
                pygame.draw.circle(screen, Circle_Color, (X_OFFSET + SQSIZE//2 + (col*SQSIZE),
                                                          Y_OFFSET + SQSIZE//2 + (row*SQSIZE)), int(0.9*CIRCLE_RADIUS))
    
    def draw(self, screen):
        # Filling the Background with Blue
        screen.fill(LIGHT_BLUE)

        # Drawing the Current Board Elements
        self.draw_board(screen)
    
    def run_GUI(self, screen):
        # Create a Flag to keep track of current state of the Application / GUI
        game_run = True

        # Main Loop
        while game_run:
            
            # Draws the Game Elements into the Screen
            self.draw(screen)

            # If we haven't reached a Final State then keep playing
            if not self.current_node.state.is_over():
                
                # Creating a new Node by using the player's tactic
                (new_node, self.clicked) = self.PLAYER_ACTION[self.current_node.state.current_player](self.current_node, self.clicked)
                
                # Updating Current Node
                self.current_node = new_node
            
                # Delays Time before showing next move
                # pygame.time.delay(300)

            # Found a Final State
            else:
                self.write_winner(screen)

            # Updates the Window
            pygame.display.update()

            """ NEEDS TO BE LOOKED AT - SOME WEIRD STUFF IS GOING ON HERE WHEN TRYING TO CLOSE THE WINDOW """
            # Main Event Loop
            for event in pygame.event.get():
                if (event.type == pygame.QUIT):
                    pygame.display.quit()
                    
        [print(move) for move in self.current_node.state.move_history]

In [45]:
new_game = Connect_Four(A_Star_terminal, random_terminal)
new_game.run_terminal()


  CURRENT BOARD 
# ------------- #
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
# ------------- #

| A* Search AI | Played 

  CURRENT BOARD 
# ------------- #
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - X |
# ------------- #

| Random AI | Played in the 3th column 

  CURRENT BOARD 
# ------------- #
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - O - - X |
# ------------- #

| A* Search AI | Played 

  CURRENT BOARD 
# ------------- #
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - X |
| - - - O - - X |
# ------------- #

| Random AI | Played in the 6th column 

  CURRENT BOARD 
# ------------- #
| - - - - - - - |
| - - - - - - - |
| - - - - - - - |
| - - - - - - O |
| - - - - - - X |
| - - - O - - X |
# ------------- #

| A* Search AI | Played 

  CURRENT BOARD 
# ----------

1

---
<div align="center">

## Graphical User Interface
</div>

---

In [15]:
class APP:
    def __init__(self):
        self.menu = "PVR"

    def run(self):
        # Initializing Window / Screen
        pygame.init()
        screen = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("Connect-4")
        ICON_IMG = pygame.image.load('./Assets/Connect-Four.png').convert_alpha()
        pygame.display.set_icon(ICON_IMG)

        # Create a Flag to keep track of current state of the Application / GUI
        run = True

        # Main Loop
        while run:

            if (self.menu == "Main"):
                screen.fill(LIGHT_BLUE)
            
            if (self.menu == "RVR"):
                new_game = Connect_Four(random_GUI, random_GUI)
                new_game.run_GUI(screen)
                self.menu = "Main"

            elif (self.menu == "PVR"):
                new_game = Connect_Four(player_GUI, random_GUI)
                new_game.run_GUI(screen)
                self.menu = "Main"
                
            # Main Event Loop
            for event in pygame.event.get():
                if (event.type == pygame.QUIT):
                    run = False
            
            # Updates the Window
            pygame.display.update()
        pygame.quit()

In [16]:
# App = APP()

In [17]:
# App.run()